In [3]:
%load_ext Cython

In [31]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp
from cython.parallel import prange
from libc.math cimport cos

cdef int num_its = 100000

def my_python_fun(int i):
    cdef int j = i**2
    cdef double k
    with nogil:
        k=my_cython_fun(j)
    return k

cdef double my_cython_fun(int i) nogil:
    cdef int j
    cdef double k=0
    for j in range(1000):
        k= cos(cos(<double>j)*k)
    return k

def parallel_fun():
    cdef int i
    cdef double k=0
    for i in prange(num_its, nogil=True):
        with gil:
            k += my_python_fun(i)
    return k
    

def serial_fun():
    cdef int i
    cdef double k=0
    for i in range(num_its):
        k+=my_python_fun(i)
    return k

In [32]:
%time parallel_fun()
%time serial_fun()

CPU times: user 2.4 s, sys: 21.8 ms, total: 2.42 s
Wall time: 732 ms
CPU times: user 2.9 s, sys: 21.8 ms, total: 2.92 s
Wall time: 2.44 s


63712.70718897356

In [20]:
%%cython 
#distutils: language = c++

from libcpp.vector cimport vector
import numpy as np
cimport numpy as np

cdef vector[np.float64_t*] array_list_to_pointers(list array_list):
    cdef vector[np.float64_t*] v
    cdef np.float64_t[:] A_view
    for A in array_list:
        A_view = A
        v.push_back(&A_view[0])
    return v

def make_pointers(array_list):
    cdef vector[np.float64_t*] v = array_list_to_pointers(array_list)

In [61]:
%%cython
import numpy as np
from cython.parallel import prange


cdef do_mult():
    cdef double[:,:] A = np.ones((1000,1000))
    cdef double[:,:] B = np.ones((1000,1000))
    cdef double[:,:] C
    # C = np.einsum("ij,jk->ik",A,B)
    with nogil:
        C = np.dot(A,B)

def do_m_parallel(int num_tries):
    cdef int i
    for i in prange(num_tries,nogil=True):
        with gil:
            do_mult()


def do_m_serial(int num_tries):
    cdef int i
    for i in range(num_tries):
        do_mult()




Error compiling Cython file:
------------------------------------------------------------
...
    cdef double[:,:] A = np.ones((1000,1000))
    cdef double[:,:] B = np.ones((1000,1000))
    cdef double[:,:] C
    # C = np.einsum("ij,jk->ik",A,B)
    with nogil:
        C = np.dot(A,B)
                 ^
------------------------------------------------------------

/home/rik/.cache/ipython/cython/_cython_magic_d0bd7a5720d8a7bc21e369c161cebf46.pyx:11:18: Calling gil-requiring function not allowed without gil

Error compiling Cython file:
------------------------------------------------------------
...
    cdef double[:,:] A = np.ones((1000,1000))
    cdef double[:,:] B = np.ones((1000,1000))
    cdef double[:,:] C
    # C = np.einsum("ij,jk->ik",A,B)
    with nogil:
        C = np.dot(A,B)
             ^
------------------------------------------------------------

/home/rik/.cache/ipython/cython/_cython_magic_d0bd7a5720d8a7bc21e369c161cebf46.pyx:11:14: Accessing Python attribute not al

In [60]:
%time do_m_serial(100)
%time do_m_parallel(100)

CPU times: user 51.5 s, sys: 383 ms, total: 51.9 s
Wall time: 52.9 s
CPU times: user 52.8 s, sys: 373 ms, total: 53.1 s
Wall time: 53.6 s


In [21]:
import numpy as np
array_list = [np.zeros(10) for _ in range(5)]
make_pointers(array_list)